In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import json
import requests
import re
from urllib.request import urlopen

In [ ]:
def get_listings(num_pages):
    all_links = set()
    url_ = "https://ws-public.interpol.int/notices/v1/red?resultPerPage=20&page={}"
    for page in range(1,num_pages+1):
        url = url_.format(str(page))
        content = BeautifulSoup(requests.get(url).content, "html.parser").text
        links = re.findall("https://ws-public.interpol.int/notices/v1/red/\d{4}-\d{5}", content)
        for link in links:
            all_links.add(link)
    return all_links

In [ ]:
#soup = BeautifulSoup(requests.get(url.format(test)).content, "html.parser")
def read_attrs(links):
    persons = []
    for url in links:
        link = urlopen(url)
        html = link.read().decode('utf-8')
        link.close()
        
        soup = BeautifulSoup(html) 
        persons.append(soup.text)
    return persons


In [ ]:
def output_json(persons, name = "data.json"):
    dict_ = {"source_code": "INTERPOL_RN",
                "source_name": "INTERPOL Red Notices",
                "source_url": "https://www.interpol.int/How-we-work/Notices/View-Red-Notices","people":[]}
    
    
    
    cols = ["name", "forename", "sex_id","date_of_birth",  "country_of_birth_id", "place_of_birth",
            "nationalities",'languages_spoken_ids', "height", "weight", "eyes_colors_id",
            "hairs_id", "distinguishing_marks"]
    other = ["entity_id","arrest_warrants", "_embedded", "_links"]
    
    for person in persons:
        curr_dict = {}
        data = json.loads(person)
        ##For more intuitive ordering of attributes
        curr_dict["ID"] = {cols[i]:data[cols[i]] for i in range(13)}
        curr_dict["other"] = {other[i]: data[other[i]] for i in range(4)}
        dict_["people"].append(curr_dict)
    
    with open(name, 'w') as f:
        json.dump(dict_, f, ensure_ascii=False, indent=2)


In [ ]:
links = get_listings(2) #get links for 2 pages
persons = read_attrs(links) #get information for everyone in the 2 pages

output_json(persons) #outpt json file default name is data.json
